In [1]:
import json
import pandas as pd

In [8]:
CDR_01_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_01.json'))
CDR_02_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_02.json'))
CDR_03_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_03.json'))
CDR_04_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_04.json'))
CDR_05_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_05.json'))
CDR_06_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_06.json'))
CDR_07_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_07.json'))
CDR_08_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_08.json'))
CDR_09_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_09.json'))
CDR_10_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_10.json'))
CDR_11_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_11.json'))
CDR_12_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_12.json'))
CDR_13_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_13.json'))
CDR_14_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_14.json'))
CDR_15_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_15.json'))
CDR_16_json = json.load(open('/workspaces/OHW_in_CDR/data/CDR/CDR_16.json'))

CDR_01_json

{'meta': {'count': 2810, 'db_response_time_ms': 93, 'page': 1, 'per_page': 25},
 'results': [{'id': 'https://openalex.org/W3080074869',
   'display_name': 'Negative emissions and the long history of carbon removal',
   'authorships': [{'author_position': 'first',
     'author': {'id': 'https://openalex.org/A5059900934',
      'display_name': 'Wim Carton',
      'orcid': 'https://orcid.org/0000-0003-3339-1514'},
     'institutions': [{'id': 'https://openalex.org/I187531555',
       'display_name': 'Lund University',
       'ror': 'https://ror.org/012a77v79',
       'country_code': 'SE',
       'type': 'education'}],
     'countries': ['SE'],
     'is_corresponding': True,
     'raw_affiliation_string': 'Lund University Centre for Sustainability Studies Lund Sweden',
     'raw_affiliation_strings': ['Lund University Centre for Sustainability Studies Lund Sweden']},
    {'author_position': 'middle',
     'author': {'id': 'https://openalex.org/A5013128967',
      'display_name': 'Adeniyi A